# with adaptation, 
https://www.swharden.com/wp/2016-07-19-realtime-audio-visualization-in-python/

In [1]:
import pyaudio
import time
import pylab
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
#import librosa, librosa.display

In [4]:
from librosa.feature import librosa_lite

C:\Users\Alicia\Desktop\Audio hardware
C:\Users\Alicia\Desktop\Audio hardware
__file__=C:\Users\Alicia\Desktop\Audio hardware\librosa\feature\librosa_lite.py | __name__=librosa.feature.librosa_lite | __package__=librosa.feature          


In [5]:
import tensorflow as tf
print(tf.__version__)
import keras

1.15.0


Using TensorFlow backend.


In [6]:
#loaded_model = keras.models.load_model('trained_mobilenet_2.h5')
loaded_model = keras.models.load_model('trained_mobilenet_2.h5')












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [7]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [12]:
class SWHear(object):
    """
    The SWHear class is made to provide access to continuously recorded
    (and mathematically processed) microphone data.
    """

    def __init__(self,device=None,startStreaming=True):
        """fire up the SWHear class."""
        print(" -- initializing SWHear")

        self.chunk = 32768 # 4096 # number of data points to read at a time
        self.rate = 44100 # time resolution of the recording device (Hz)

        # for tape recording (continuous "tape" of recent audio)
        self.tapeLength=2 #seconds
        self.tape=np.zeros(self.rate*self.tapeLength) 
        # (np.empty(44100*2)*np.nan).shape
        # print(self.tape.shape) # (88200,)


        self.p=pyaudio.PyAudio() # start the PyAudio class
        if startStreaming:
            self.stream_start()

    ### LOWEST LEVEL AUDIO ACCESS
    # pure access to microphone and stream operations
    # keep math, plotting, FFT, etc out of here.

    def stream_read(self):
        """return values for a single chunk"""
        data = np.frombuffer(self.stream.read(self.chunk),dtype=np.int16)
        #print(data)
        return data

    def stream_start(self):
        """connect to the audio device and start a stream"""
        print(" -- stream started")
        self.stream=self.p.open(format=pyaudio.paInt16,channels=1,
                                rate=self.rate,input=True,
                                frames_per_buffer=self.chunk)

    def stream_stop(self):
        """close the stream but keep the PyAudio instance alive."""
        if 'stream' in locals():
            self.stream.stop_stream()
            self.stream.close()
        print(" -- stream CLOSED")

    def close(self):
        """gently detach from things."""
        self.stream_stop()
        self.p.terminate()

    ### TAPE METHODS
    # tape is like a circular magnetic ribbon of tape that's continously
    # recorded and recorded over in a loop. self.tape contains this data.
    # the newest data is always at the end. Don't modify data on the type,
    # but rather do math on it (like FFT) as you read from it.

    def tape_add(self):
        """add a single chunk to the tape."""
        self.tape[:-self.chunk]=self.tape[self.chunk:]
        self.tape[-self.chunk:]=self.stream_read()

    def tape_flush(self):
        """completely fill tape with new data."""
        readsInTape=int(self.rate*self.tapeLength/self.chunk) 
        print(" -- flushing %d s tape with %dx%.2f ms reads"%\
                  (self.tapeLength,readsInTape,self.chunk/self.rate))
        for i in range(readsInTape):
            self.tape_add()
            
            
    def tape_forever(self,plotSec=.25):
        t1=0
        try:
            while True:
                self.tape_add()
                
                # add stuff here
                # self.tape_logspec() 
                # add stuff here
                # self.tape_spec() = librosa.feature.melspectrogram(self.tape_add(), sr = self.rate, n_mels=128)
                # self.tape_logspec() = librosa.amplitude_to_db(self.tape_spec())
                # melspec = librosa.feature.melspectrogram(data.astype(float), sr=RATE, n_mels=128)
                # log_S = librosa.amplitude_to_db(melspec)
                
                if (time.time()-t1)>plotSec:
                    t1=time.time()
                    print()
                    self.tape_plot()
                    self.tape_processing()
                    self.tape_inferencing()
    
        except:
            print(" ~~ exception (keyboard?)")
            raise
            return

    def tape_plot(self,saveAs="03.png"):
        """plot what's in the tape."""        
        plt.plot(np.arange(len(self.tape))/self.rate,self.tape) 
        plt.axis([0,self.tapeLength,-2**16/2,2**16/2])
        if saveAs:
            t1=time.time()
            plt.savefig(saveAs,dpi=70)
            print("waveform plotting saving took:            %.02f ms"%((time.time()-t1)*1000))
        else:
            plt.show()
            print() #good for IPython
        plt.close('all') 
        
    

    # add stuff here
    def tape_processing(self,saveAs="04.png"):
        """plot log mel spec content in the tape"""
        # print("running tape processing")
        # print(self.tape)
        # print(self.tape.shape) # (88200,)
#         self.melspec = librosa.feature.melspectrogram(self.tape/65536, sr=self.rate, n_mels=128)
#         self.log_S = librosa.amplitude_to_db(self.melspec)
        self.melspec = librosa_lite.melspectrogram(self.tape/65536, sr= self.rate, n_mels=128)
        self.log_S = librosa_lite.amplitude_to_db(self.melspec)
        # print(self.log_S.shape) # (128,173)
        
        fig, ax = plt.subplots(figsize=(12,4))
        cax = ax.matshow(
            self.log_S.astype("int16"),
            interpolation="nearest",
            aspect="auto",
            cmap=plt.cm.afmhot,
            origin="lower",
        )
        fig.colorbar(cax)
        plt.title("mel Spectrogram")
#         plt.figure()
#         librosa.display.specshow(self.log_S, sr= self.rate, x_axis='time', y_axis='mel')
#         plt.colorbar(format='%+0.2f dB')
#         plt.tight_layout()
        if saveAs: 
            t2=time.time()
            plt.savefig(saveAs, dpi=70)
            print("log mel spectrogram plotting saving took: %.02f ms"%((time.time()-t2)*1000))
        else: 
             plt.show()
             print()
        plt.close('all')

#         # melspec = librosa.feature.melspectrogram(data.astype(float), sr=RATE, n_mels=128)
#         # log_S = librosa.amplitude_to_db(melspec)
    
#     plt.figure()
#     librosa.display.specshow(log_S, sr=RATE, x_axis='time', y_axis='mel')
#     plt.title('Log mel spectrogram')
#     plt.colorbar(format='%+02.0f dB')
#     plt.tight_layout()
#     plt.savefig("04.png",dpi=80)
#     plt.close('all')

    def tape_inferencing(self):
        """inferencing content in the tape"""
        # print("running tape inferencing")

        start_index_x = (224-128)//2
        # start_index_x = 48
        end_index_x = (224-128)//2 + 128
        # end_index_x = 176
        start_index_y = (224-173)//2
        # start_index_y = 25
        end_index_y = (224-173)//2 + 173
        # end_index_y = 198
        # print(start_index_x, end_index_x, start_index_y, end_index_y)

        # self.test_recorded_data = np.empty(shape=(1,224,224,3))
        # for i in range(number):
        # mean = np.mean(splits_pad[i][self.log_S]) # shape is alr (128,173)
        # data_squeeze = np.squeeze(train_all[i])# (128,173,1) to (128,173) 
        
        self.mean = np.mean(self.log_S) # shape is alr (128,173)
        self.spectrogram = self.mean*np.ones(shape = (224,224,3))
        for j in range(3):
            self.spectrogram[start_index_x:end_index_x, start_index_y:end_index_y,j] = self.log_S

        #self.test_recorded_data = self.spectrogram
        
        t3 = time.time()
        self.predictions = loaded_model.predict(np.expand_dims(self.spectrogram, axis=0))
        print("inferencing took:                         %.02f ms"%((time.time()-t3)*1000))
    
        # self.predictions = loaded_model.predict(self.spectrogram)
        
        # print(np.argmax(self.predictions), self.predictions)
        
        
        classidx = np.argmax(self.predictions[0]) 
        
        def classify(i):
            switcher = {
                0: "fall",
                1: "cough",
                2: "shout",
                3: "speech"
            }
            return switcher.get(i, "Invalid class")
        
        classlab = self.predictions[0]
        
        thresh = 0.60
        if classlab[0] < thresh and classlab[1] < thresh and classlab[2] < thresh and classlab[3] < thresh:
            print("sound class:                              ambience sound")
        else:
            print("sound class:                             ", classify(classidx), classlab[classidx])
    
        
        # print("print all for checking:                  ", classidx, classlab)
        
        print(np.argmax(self.predictions), self.predictions)
        

y = np.ones(shape=(224,224,3))
print(y.shape)
y = np.expand_dims(y, axis=0)
print(y.shape)

In [13]:
if __name__=="__main__":
    ear=SWHear()
    ear.tape_forever()
    ear.close()
    print("DONE")

 -- initializing SWHear
 -- stream started

waveform plotting saving took:            116.69 ms
log mel spectrogram plotting saving took: 166.55 ms
inferencing took:                         161.57 ms
sound class:                              ambience sound
0 [[0.40823293 0.4032832  0.00706329 0.18142055]]

waveform plotting saving took:            90.78 ms
log mel spectrogram plotting saving took: 169.54 ms
inferencing took:                         98.74 ms
sound class:                              ambience sound
3 [[0.32920688 0.28139323 0.0129463  0.37645364]]

waveform plotting saving took:            74.75 ms
log mel spectrogram plotting saving took: 160.57 ms
inferencing took:                         131.65 ms
sound class:                              fall 0.65746
0 [[0.65746    0.20690924 0.00788753 0.12774329]]

waveform plotting saving took:            90.81 ms
log mel spectrogram plotting saving took: 158.54 ms
inferencing took:                         107.71 ms
sound class:  

KeyboardInterrupt: 

(np.empty(44100*2)*np.nan).shape